<a href="https://colab.research.google.com/github/Ksyusha7650/ML_1lab/blob/main/Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Лабораторная работа 1**

импорт библиотек

# подготовка массива изображений

In [22]:
import zipfile
import os
import numpy as np
import cv2
from google.colab import drive
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D
from tensorflow.keras.models import Model

подключение Гугл диска, извлечение картинок

In [5]:
drive.mount('/content/drive', force_remount=True)
zip_path = '/content/drive/My Drive/CarDatasets.zip'
extract_path = "./images"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Mounted at /content/drive


загрузка картинок в массив

In [6]:
image_paths = []
for root, _, files in os.walk(extract_path):
    for file in files:
        if file.endswith(('.png', '.jpg', '.jpeg')):
            image_paths.append(os.path.join(root, file))

image_size = (128, 128)  # Размер изображений для модели
image_data = []

for img_path in image_paths:
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Преобразуем BGR -> RGB
    img = cv2.resize(img, image_size)  # Изменяем размер
    img = img_to_array(img) / 255.0  # Нормализация
    image_data.append(img)

# Преобразуем в NumPy массив
image_data = np.array(image_data)
print(f'Загружено {image_data.shape[0]} изображений размером {image_size}')

Загружено 16185 изображений размером (128, 128)


# оптимизатор AdaSmooth на основе InceptionV3

In [24]:
class AdaSmooth(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.001, name="AdaSmooth", **kwargs):
        super().__init__(name, **kwargs)
        # Инициализируем learning_rate через hyperparameter API
        self._set_hyper("learning_rate", learning_rate)

    def _resource_apply_dense(self, grad, var, apply_state=None):
        lr = self._get_hyper("learning_rate")
        var.assign_sub(lr * grad)

    def _resource_apply_sparse(self, grad, var, indices, apply_state=None):
        lr = self._get_hyper("learning_rate")
        tf.compat.v1.scatter_sub(var, indices, lr * grad)

    def get_config(self):
        config = super().get_config()
        config.update({
            "learning_rate": self._serialize_hyperparameter("learning_rate")
        })
        return config

In [25]:
base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
base_model.trainable = False  # Замораживаем базовую модель

# Добавляем кастомные слои
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
output_layer = Dense(1, activation='sigmoid')(x)

model = Model(inputs=base_model.input, outputs=output_layer)

# Компилируем модель с оптимизатором AdaSmooth
optimizer = AdaSmooth(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Выводим архитектуру модели
model.summary()

ValueError: Argument `learning_rate` should be float, or an instance of LearningRateSchedule, or a callable (that takes in the current iteration value and returns the corresponding learning rate value). Received instead: learning_rate=AdaSmooth